In [ ]:
# Import python packages
import tangram as tg
from anndata import AnnData
import sys, os
sys.path.insert(0, "FISHscale")
from FISHscale.utils import dataset
import scanpy as sc
sc.settings.verbosity = 0
sc.set_figure_params(dpi = 450, dpi_save = 450)
import numpy as np
from scipy.spatial.distance import pdist
from scipy.spatial import KDTree
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb, hex2color
import pandas as pd

# Set plotting settings
import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = "none"
mpl.rcParams['legend.labelcolor'] = "black"
mpl.rcParams['text.color'] = "black"

if not os.path.exists('figures'):
   os.makedirs('figures')

In [ ]:
# Read spatial data
d = dataset.Dataset("Data/LBEXP20211113_EEL_HE_5w_970um_RNA_transformed_assigned.parquet",
                       gene_label = 'decoded_genes', 
                       x_label = 'r_transformed', 
                       y_label = 'c_transformed', 
                       pixel_size='0.27 micrometer', 
                       other_columns=['Brain'],
                       z = 970,
                       reparse=False)
d.set_working_selection("Brain")

In [ ]:
# Creating AnnData object from fishscale spatial data

# Make hexagonal bins, the output can be used to create adata object
# Returns pandas count dataframe to df_hex and spatial coordinates (x, y) to a list variable
df_hex, coordinates = d.hexbin_make(spacing=50, min_count=10)

# Initialize variables
coordinates_to_keep = []
count_df = df_hex.T
spots_to_remove = []
x, y = [], []

# The spatial data contains part (hindbrain) which doesn't occur in our mistr data 
# Using the spatial coordinates (x, y) we are able to select the spots we are interested in 
for i, spot in enumerate(count_df.index):
    if coordinates[i][0] < 1800 and coordinates[i][1] < 4300:
        spots_to_remove.append(spot)
    else:
        coordinates_to_keep.append([coordinates[i][0]*-1,coordinates[i][1]])
        x.append(coordinates[i][0]*-1)
        y.append(coordinates[i][1])

# Remove the unnecessary rows from df based on the spots_to_remove index list
count_df = count_df.drop(spots_to_remove)

# Initialize anndata with count data, and add the spatial cordinates 
adata_spatial = AnnData(count_df.to_numpy(), obsm={"spatial": np.array(coordinates_to_keep)})
adata_spatial.var.index = np.array(count_df.columns)
adata_spatial.obs.index = np.array(count_df.index)
adata_spatial.obs["x"] = x
adata_spatial.obs["y"] = y

# Plot results after removing the parts not occuring in our single cell data
with plt.rc_context({"figure.figsize": [3.5, 2.2],   "figure.dpi": 400}):
    xs = adata_spatial.obs.x.values
    ys = adata_spatial.obs.y.values
    plt.axis("off")
    plt.scatter(xs, ys, s=0.8, color="#D3D3D3");
    plt.savefig("figures/raw_spatial_data.png", dpi=450, bbox_inches='tight')
    plt.savefig("figures/raw_spatial_data.svg", dpi=450, bbox_inches='tight')

In [ ]:
# Tangram plotting function inverts the y axis, and there is no parameter to control this
# Therefore this function can be used to invert the y axis before passing the data to tangram plotting functions
# and when tangram inverts the axis again, the plot is displayed correctly

def invert_y_axis_for_tangram(adata):
    counter, inverted_coordinates = 0, []
    # Looping over the coordinatas, and multiplying the y-axis coordinates by 1 and append the updated coordinates to list
    for i in range(len(adata.obsm["spatial"])):
        inverted_coordinates.append((adata.obsm["spatial"][i][0],adata.obsm["spatial"][i][1]*-1))

    # Store the updated coordinates in adata_spatial object
    adata.obsm["spatial"] = np.array(inverted_coordinates)
    return adata

# Calling the function that invers y axis, both with the full and subsetted spatial data
adata_spatial = invert_y_axis_for_tangram(adata_spatial)

In [ ]:
# This code is edited version of tangram plot_genes_sc function
# The original function contained some shortcommings,
# and therefore a customized version of it was used to plot mesured / projected genes on space
# The original function allows plotting measured and projected expression side by side,
# whereas this function plots either measured or projected genes
# Other changes: possibility to save the figure and some additional parameters that serves better our purposes

from scipy.sparse.csr import csr_matrix
from scipy.sparse.csc import csc_matrix
from matplotlib.gridspec import GridSpec
import pylab as pl

sc.set_figure_params(dpi_save = 500)
sc.set_figure_params(scanpy=True, fontsize=10)

def construct_obs_plot(df_plot, adata, perc=0, suffix=None):
    # clip
    df_plot = df_plot.clip(df_plot.quantile(perc), df_plot.quantile(1 - perc), axis=1)

    # normalize
    df_plot = (df_plot - df_plot.min()) / (df_plot.max() - df_plot.min())

    if suffix:
        df_plot = df_plot.add_suffix(" ({})".format(suffix))
    adata.obs = pd.concat([adata.obs, df_plot], axis=1)

def convert_adata_array(adata):
    if isinstance(adata.X, csc_matrix) or isinstance(adata.X, csr_matrix):
        adata.X = adata.X.toarray()
        
def tangram_plot_genes_custom(adata, genes=[], x="x",y = "y",spot_size=None, save=False,scale_factor=None, ncols=4, cmap="inferno",
                              perc=0,alpha_img=1.0,bw=False,return_figure=False, plot_measured_spatial=False, show=True):
    # construct df_plot
    data = []
    
    # remove df_plot in obs
    if plot_measured_spatial == False:
        adata.obs.drop(["{} (projected gene expression)".format(gene) for gene in genes], inplace=True, errors="ignore", axis=1)
        adata.var.index = [g.upper() for g in adata.var.index]
        adata.obs.drop(["{} (projected gene expression)".format(gene) for gene in genes], inplace=True, errors="ignore", axis=1)
        
        df = pd.DataFrame(data=np.array(adata[:, genes].X), columns=genes, index=adata.obs.index)
        construct_obs_plot(df, adata, perc=perc, suffix="projected gene expression")
            
    else:
        adata.obs.drop(["{} (measured gene expression)".format(gene) for gene in genes], inplace=True, errors="ignore", axis=1)
        
        # prepare adatas
        convert_adata_array(adata)
        adata.var.index = [g.upper() for g in adata.var.index]
        adata.obs.drop(["{} (measured gene expression)".format(gene) for gene in genes],inplace=True,errors="ignore",axis=1,)
        for ix, gene in enumerate(genes):
            if gene not in adata.var.index:
                data.append(np.zeros_like(np.array(adata[:, 0].X).flatten()))
            else:
                data.append(np.array(adata[:, gene].X).flatten())

        df = pd.DataFrame(
            data=np.array(data).T, columns=genes, index=adata.obs.index,
        )
        construct_obs_plot(df, adata, suffix="measured gene expression")

    fig = plt.figure(figsize=(7, len(genes) * 3.5))
    gs = GridSpec(len(genes), 2, figure=fig)
    
    #non visium data
    if 'spatial' not in adata.obsm.keys():
        #add spatial coordinates to obsm of spatial data 
        if plot_measured_spatial == True:
            coords = [[x,y] for x,y in zip(adata.obs[x].values,adata.obs[y].values)]
            adata.obsm['spatial'] = np.array(coords)
        else:
            coords = [[x,y] for x,y in zip(adata.obs[x].values,adata.obs[y].values)]
            adata.obsm['spatial'] = np.array(coords)

    if ("spatial" not in adata.uns.keys()) and (spot_size==None and scale_factor==None):
        raise ValueError("Spot Size and Scale Factor cannot be None when ad_sp.uns['spatial'] does not exist")
        
    gene_list=[]
    for gene in genes:
        if plot_measured_spatial==True:       
            gene_list.append("{} (measured gene expression)".format(gene))          
        else:
            gene_list.append("{} (projected gene expression)".format(gene))
        
    if plot_measured_spatial == True:
        sc.pl.spatial(adata, spot_size=spot_size, scale_factor=scale_factor, color=gene_list,
            frameon=False, show=show, cmap=cmap, alpha_img=alpha_img, bw=bw, ncols=ncols, colorbar_loc=None, save=save)
    else: 
        #key = dict_keys[i+j]
        #sc.pl.spatial(adata, spot_size=spot_size, scale_factor=scale_factor, color=["{} (predicted)".format(gene)],
        sc.pl.spatial(adata, spot_size=spot_size, scale_factor=scale_factor, color=gene_list, 
            frameon=False, show=show, cmap=cmap, alpha_img=alpha_img, bw=bw, ncols=ncols, colorbar_loc=None,save=save )
    
    if show == True:
        a = np.array([[0,1]])
        pl.figure(figsize=(0.14, 4.5))
        img = pl.imshow(a, cmap="YlGnBu")
        pl.gca().set_visible(False)
        cax = pl.axes([0.1, 0.2, 0.8, 0.6])
        pl.colorbar(orientation="vertical", cax=cax)
        pl.savefig("figures/YlGnBu_colorbar_1.png", dpi=450, bbox_inches='tight')

        a = np.array([[0,1]])
        pl.figure(figsize=(0.14, 1.8))
        img = pl.imshow(a, cmap="YlGnBu")
        pl.gca().set_visible(False)
        cax = pl.axes([0.1, 0.2, 0.8, 0.6])
        pl.colorbar(orientation="vertical", cax=cax)
        pl.savefig("figures/YlGnBu_colorbar_2.png", dpi=450, bbox_inches='tight')
    
    # remove df_plot in obs
    if plot_measured_spatial == True:
        adata.obs.drop(["{} (measured gene expression)".format(gene) for gene in genes], inplace=True, errors="ignore", axis=1)
    else:
        adata.obs.drop(["{} (projected gene expression)".format(gene) for gene in genes], inplace=True, errors="ignore", axis=1)
          
    if return_figure==True:
        return fig

In [ ]:
# Plot the gene expression in spatial data
for gene in ['OTX2', 'WNT1', 'FGF8', 'FGF17', 'FGF18']:  
    tangram_plot_genes_custom(adata_spatial, genes=[gene],spot_size=50,
                          scale_factor=0.1, perc = 0.001, return_figure=False, cmap="YlGnBu", plot_measured_spatial=True, save="_measured_png/_"+gene+"_measured_expression.png", show=False)